# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

__Мягкий дедлайн: 24.09 23:59__   
__Жесткий дедлайн: 27.09 23:59__

### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в почте или поисковой строке. За дополнительные баллы полученную систему нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex) или аналогов. В этой домашке вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей: основной и бонусной. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание и штрафы

Максимально допустимая оценка за работу — 15 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по __одному__ баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код.

При сдаче зададания в anytask вам будет необходимо сдать весь код, а если вы возьметесь за бонусную часть, то еще отчет и видео с демонстрацией вашего UI. За основную часть можно получить до __10-ти__ баллов, а за бонусную – до __5-ти__ баллов.

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [1]:
import pandas as pd

emails = pd.read_csv('emails.csv')
len(emails)

517401

Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (2 балла).__ Очистите корпус текстов по вашему усмотрению и объясните свой выбор. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст.

In [ ]:
import email
import re
def parse(text):
    text = email.message_from_string(text).get_payload() # убираем хеадеры

    pattern = re.compile(
    r"-----\s*Original Message\s*-----[\s\S]*?(?=\n\n|\Z)", 
    re.IGNORECASE
    ) 
    text = re.sub(pattern, "", text) # убираем original message вставки

    text = re.sub(r"\n([_*=\-~])\1{4,}[\s\S]*$", "", text).strip() # убираем всякие шняги внизу аля подписи

    text = re.sub(
        r"(?:^|\n)(?:\s*(From|To|Cc|cc|Subject):.*\n?)+", 
        "\n", 
        text, 
        flags=re.IGNORECASE
    ).strip() # убираем контакты кто-куда
    text = re.sub(r"http[s]?://\S+|www\.\S+", "", text)
    text = re.sub(r"\n+", "\n", text).strip() # убираем лишние переносы строк

    return text

Выбор объясняю тем, чтобы оставить только само содержание без метанформации и подписей (причем сделать парсер максимально простым, чтобы быстро работал). Вообще в продакшн сетапе я возможно бы запаралися и использовал легковесную LLM, чтобы она спарсила при помощи хорошего систем промпта, так как регулярками все равно не покроешь все случаи. 

In [7]:
for x in emails['message'].sample(3, random_state=42):
    print(x[:400])
    print("AFTER")
    print(parse(x)[:400])
    print('='*50)

Message-ID: <21013688.1075844564560.JavaMail.evans@thyme>
Date: Tue, 29 Aug 2000 01:26:00 -0700 (PDT)
From: sara.shackleton@enron.com
To: william.bradford@enron.com
Subject: Re: Credit Derivatives
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Sara Shackleton
X-To: William S Bradford
X-cc: 
X-bcc: 
X-Folder: \Sara_Shackleton_Dec2000_June2001_1\
AFTER
Bill:  Thanks for the info.   I also spoke with Jeff about how 
EnronCredit.com Ltd. was going to work since Dennis O'Connell (London lawyer) 
is responsible for that group.  Maybe you will be able to clarify which of 
Jeff's "positions" will be hedges and which will be backed to EnronCredit.  
Maybe Rod will be handling most of Jeff's credit.  I'd appreciate an update.  
Sara
	William S Bradford

Message-ID: <22688499.1075854130303.JavaMail.evans@thyme>
Date: Mon, 24 Apr 2000 05:43:00 -0700 (PDT)
From: pat.clynes@enron.com
To: aimee.lannou@enron.com
Subject: Meter #1591 Lamay Gaslift
C

In [8]:
emails['processed'] = emails['message'].apply(parse)

In [9]:
for x in emails['processed'].sample(10, random_state=42):
    print(x)

Bill:  Thanks for the info.   I also spoke with Jeff about how 
EnronCredit.com Ltd. was going to work since Dennis O'Connell (London lawyer) 
is responsible for that group.  Maybe you will be able to clarify which of 
Jeff's "positions" will be hedges and which will be backed to EnronCredit.  
Maybe Rod will be handling most of Jeff's credit.  I'd appreciate an update.  
Sara
	William S Bradford
	08/29/2000 07:24 AM
Nelson/LON/ECT@ECT
Sara,
Please contact either Paul Radous or me on credit derivatives in the U.S.  
Rod Nelson is the lead credit support for EnronCredit.com and should also be 
available in London, if necessary.  I am not aware of these recent trades but 
I am having lunch with Jeff Kinneman on Thursday to discuss among other 
things Credit support for his business.  
It does concern me that we would offer to provide collateral DLJ without 
Treasury's approval.
Bill
I am seeing more and more credit derivatives.  The trades originating in 
Houston are coming from Jeff Kin

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат для финального пользователя будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы заметите, что из-за этого падает качество самого текса, то можете удалить все небуквенные символы на этапе токенизации.

In [ ]:
def tokenize(text: str, clean_non_letters: bool = False, lowercase: bool = True) -> List[str]:
    if lowercase:
        text = text.lower()

    if not clean_non_letters:
        return [t for t in text.split() if t]

    return re.findall(r"[a-zA-Zа-яА-ЯёЁ]+", text)


## Общая схема решения

Мы хотим сделать систему, которая будет ускорять набор текста, советуя подходящие продолжения. Для подсказки следующего слова мы будем использовать n-граммную модель. Так как n-граммная модель работает с целыми словами, а советы мы хотим давать в риал-тайме даже когда слово еще не дописано, сперва надо научиться дополнять слово до целого.

## Дополнение слова

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова не больше чем за $O(n + mk)$, где $m$ - число подходящих слов, а $k$ – длина суффикса.

__Задание 2 (2 балла).__ Допишите префиксное дерево для поиска слов по префиксу.

In [ ]:
from typing import List

class PrefixTreeNode:
    def __init__(self):
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False

class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()
        
        for word in vocabulary:
            self.insert(word)

    def insert(self, word: str) -> None:
        node = self.root
        for c in word:
            if c not in node.children:
                node.children[c] = PrefixTreeNode()
            node = node.children[c]
        node.is_end_of_word = True


    def search_prefix(self, prefix) -> List[str]:
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """

        node = self.root
        for c in prefix:
            if c not in node.children:
                return []
            node = node.children[c]

        result = []
        self.dfs(node, prefix, result)
        return result
    
    def dfs(self, node: PrefixTreeNode, prefix: str, result: List[str]) -> None:
        if node.is_end_of_word:
            result.append(prefix)
        for c, child in node.children.items():
            self.dfs(child, prefix + c, result)

In [11]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его __встречаемости в корпусе__.

__Задание 3 (2 балла).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [24]:
from collections import Counter
from typing import List, Tuple

class WordCompletor:
    def __init__(self, corpus):
        """
        corpus: list – корпус текстов
        """
        flat_corpus = [word for doc in corpus for word in doc]
        self.word_counts = Counter(flat_corpus)
        self.total_count = sum(self.word_counts.values())
        vocabulary = list(self.word_counts.keys())
        self.total_count = sum(self.word_counts.values())
        self.prefix_tree = PrefixTree(vocabulary)

    def get_words_and_probs(self, prefix: str) -> Tuple[List[str], List[float]]:
        """
        Возвращает список слов, начинающихся на prefix,
        с их вероятностями (нормировать ничего не нужно)
        """
        words = self.prefix_tree.search_prefix(prefix)
        probs = [self.word_counts[w] / self.total_count for w in words]
        return words, probs

In [25]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему следующее слово (или несколько) с учетом дописанного. Для этого мы воспользуемся n-граммной моделью.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

$P(w_i \mid w_{i-1}, \dots, w_{i-n})$ оценивается по частоте встречаемости n-граммы.   

__Задание 4 (2 балла).__ Напишите класс для n-граммной модели. Никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [47]:

from collections import defaultdict, Counter

class NGramLanguageModel:
    def __init__(self, corpus, n):
        self.n = n
        self.model = defaultdict(Counter)
        for sent in corpus:
            for i in range(len(sent) - n):
                context = tuple(sent[i:i+n])
                next_word = sent[i+n]
                self.model[context][next_word] += 1

    def get_next_words_and_probs(self, prefix):
        if len(prefix) < self.n:
            return [], []
        context = tuple(prefix[-self.n:])
        counter = self.model.get(context)
        if not counter:
            return [], []
        total = sum(counter.values())
        words, probs = [], []
        for w, c in counter.items():
            words.append(w)
            probs.append(c / total)
        return words, probs


In [48]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (2 балла).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. После этого можно добавить опцию генерации нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [49]:
from typing import Union, List
import random
class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text: Union[str, list], n_words=3, n_texts=1) -> list[list[str]]:
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """

        suggestions = []

        tokens = text.split() if isinstance(text, str) else list(text)
        if not tokens:
            return []

        last_word = tokens[-1]
        completions, comp_probs = self.word_completor.get_words_and_probs(last_word)
        if not completions:
            return []  
        completed_word = completions[0]

        base_prefix = tokens[:-1] + [completed_word]

        suggestions: List[List[str]] = [[] for _ in range(n_texts)]

        for j in range(n_texts):
            current_text = list(base_prefix)
            suggestions[j].append(completed_word)

            for _ in range(n_words):
                next_words, probs = self.n_gram_model.get_next_words_and_probs(current_text)
                if not next_words:
                    break

                if n_texts == 1:
                    best_idx = max(range(len(next_words)), key=lambda i: probs[i])
                    next_word = next_words[best_idx]
                else:
                    next_word = random.choices(next_words, weights=probs, k=1)[0]

                current_text.append(next_word)
                suggestions[j].append(next_word)

        return suggestions


In [ ]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

In [56]:
emails['tokenized'] = emails['processed'].apply(tokenize, args=(True, True))

In [ ]:
emails['tokenized_spaces'] = emails['processed'].apply(tokenize, args=(False, True))

In [60]:
emails['tokenized'] 

0                                 [here, is, our, forecast]
1         [traveling, to, have, a, business, meeting, ta...
2                           [test, successful, way, to, go]
3         [randy, can, you, send, me, a, schedule, of, t...
4                         [let, s, shoot, for, tuesday, at]
                                ...                        
517396    [this, is, a, trade, with, oil, spec, hedge, n...
517397    [some, of, my, position, is, with, the, albert...
517398    [morning, john, i, m, still, working, on, the,...
517399    [analyst, rank, stephane, brodeur, chad, clark...
517400    [i, think, the, ymca, has, a, class, that, is,...
Name: tokenized, Length: 517401, dtype: object

In [59]:
emails['tokenized_spaces']

0                                 [here, is, our, forecast]
1         [traveling, to, have, a, business, meeting, ta...
2                       [test, successful., way, to, go!!!]
3         [randy,, can, you, send, me, a, schedule, of, ...
4                  [let's, shoot, for, tuesday, at, 11:45.]
                                ...                        
517396    [this, is, a, trade, with, oil-spec-hedge-ng, ...
517397    [some, of, my, position, is, with, the, albert...
517398    [2, morning, john,, i'm, still, working, on, t...
517399    [analyst, rank, stephane, brodeur, 1, chad, cl...
517400    [i, think, the, ymca, has, a, class, that, is,...
Name: tokenized_spaces, Length: 517401, dtype: object

In [111]:
word_completor = WordCompletor(emails['tokenized'] )

n_gram_model = NGramLanguageModel(corpus=emails['tokenized'] , n=1)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

In [112]:
query = "hello bro".split()
text_suggestion.suggest_text(query, n_words=5, n_texts=1)

[['bro', 'confirm', 'that', 'the', 'company', 's']]

In [113]:
word_completor = WordCompletor(emails['tokenized_spaces'] )

n_gram_model = NGramLanguageModel(corpus=emails['tokenized_spaces'] , n=1)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

In [114]:
query = "hello bro".split()
text_suggestion.suggest_text(query, n_words=5, n_texts=1)

[['bro', 'confirm', 'that', 'the', 'following', 'the']]

In [130]:
query = "cringe".split()
text_suggestion.suggest_text(query, n_words=3, n_texts=3)

[['cringe', 'at', 'carolina.', 'tony'],
 ['cringe', 'out', 'of', 'any'],
 ['cringe', 'out', 'why!', 'vince']]

## Бонусная часть: Добавляем UI

Запускать ячейки в юпитере – это хорошо, но будет лучше, если вашим решением действительно можно будет пользоваться. Для этого вам предлагается добавить полноценных User Interface. Мы рекомендуем использовать для этого [reflex](https://github.com/reflex-dev/reflex). Это Python библиотека для создания web-интерфейсом с очень богатым функционалом.

Ваша задача – сделать поле для текстового ввода, при наборе текста в котором будут появляться подсказки в реальном времени. Продумайте, как пользователь будет выбирать подсказки, сколько продолжений рекомендавать и так далее. В общем, должно получиться красиво и удобно. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным. 

За это задание можно получить до __5-ти бонусных баллов__ в зависимости о того, насколько хорошо и удобно у вас получилось. При сдаче задания прикрепите небольшой __отчет__ (полстраницы) с описанием вашей системы, а также __видео__ (1-2 минуты) с демонстрацией работы интерфейса.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.